In [1]:
from __future__ import print_function

"""
A script to run cqed_rhf method on the formaldehyde molecule in a cc-pVDZ basis set.
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley"]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
from helper_cqed_rhf import *
from helper_cs_cqed_cis import *
from helper_cis import *
from matplotlib import pyplot as plt
from psi4.driver.procrouting.response.scf_response import tdscf_excitations

# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
#psi4.core.set_output_file('output.dat', False)

numpy_memory = 2


In [4]:

# rhf/cc-pVDZ optimized geometry of formaldehyde
molstr = """

0 1
O 0.0000000000 0.0000000000 5.91268220e-01
C 0.0000000000 0.0000000000 -5.90400099e-01
H 0.0000000000 9.32184336e-01 -1.17703144e+00
H 0.0000000000 -9.32184336e-01 -1.17703144e+00
no_reorient
symmetry c1
"""

# options dict
options_dict = {'basis': 'cc-pVDZ',
               'save_jk': True, 
               'scf_type': 'pk',
               'e_convergence' : 1e-10,
               'd_convergence' : 1e-10}

psi4.set_options(options_dict)
mol = psi4.geometry(molstr)


rhf_energy, wfn = psi4.energy("scf/cc-pVDZ", molecule=mol, return_wfn=True)
print(rhf_energy)
    
# calculate the excited-state energies and save them to a dictionary called 'res'
res = tdscf_excitations(wfn, states=5, triplets = "NONE", tda=True)


# parse res for excitation energies
psi4_excitation_e = [r["EXCITATION ENERGY"] for r in res]

psi4_tdm = [r["ELECTRIC DIPOLE TRANSITION MOMENT (LEN)"] for r in res]




# run helper_cis()
#cis_e, cis_c, cis_mu, tdm = cis(molstr, options_dict)

print("psi4 dE")
print(psi4_excitation_e[0] * 27.211, psi4_excitation_e[1] * 27.211, psi4_excitation_e[2] * 27.211, psi4_excitation_e[3] * 27.211, psi4_excitation_e[4] * 27.211)
print("psi4 transition dipole moments")
print(psi4_tdm)

-113.87722217661789
psi4 dE
4.726755504001224 10.286294990651296 10.35053938898499 10.556573151383908 11.907679441387744
psi4 transition dipole moments
[array([ 1.15930044e-15, -3.06690717e-16, -3.07492898e-16]), array([ 4.57978222e-02,  2.94977812e-14, -1.69465683e-15]), array([ 1.20416738e-15, -9.95224321e-01, -2.93178642e-13]), array([-5.70838140e-17,  5.86910855e-13, -7.98961702e-01]), array([-4.26152588e-15, -7.81841273e-17, -1.13463960e-14])]


Compute CQED-CIS ground state for the $\lambda_z$ and $\lambda_{yz}$ cases.

In [6]:
# magnitude of coupling
l = 0.2

# photon frequency
om = 10.4/27.211

no_lam = np.array([0., 0., 0.])
lam_z = np.array([0, 0, l])
lam_yz = np.array([0, l/np.sqrt(2), l/np.sqrt(2)])
    
# cqed-rhf results 
yz_cqed_rhf_dict = cqed_rhf(lam_yz,molstr, options_dict)
z_cqed_rhf_dict = cqed_rhf(lam_z, molstr, options_dict)

# cqed-cis results 
yz_cqed_cis_dict = cs_cqed_cis(lam_yz, om, molstr, options_dict)
z_cqed_cis_dict = cs_cqed_cis(lam_z, om, molstr, options_dict)


print("cqed_cis energy relative to rhf for \lambda_z:", z_cqed_cis_dict['CQED-CIS ENERGY'][0]-rhf_energy)
print("cqed_cis energy relative to rhf for \lambda_yz:", yz_cqed_cis_dict['CQED-CIS ENERGY'][0]-rhf_energy)





Start SCF iterations:

Canonical RHF One-electron energy = -218.2835959999194131
CQED-RHF One-electron energy      = -217.8061796768820386
Nuclear repulsion energy          = 31.7803657982661569
Dipole energy                     = 0.0142623507570482
SCF Iteration   1: Energy = -113.7228685077951269   dE = -1.13723E+02   dRMS = 2.16876E-03
SCF Iteration   2: Energy = -113.7282528209207300   dE = -5.38431E-03   dRMS = 8.46355E-04
SCF Iteration   3: Energy = -113.7286889666430909   dE = -4.36146E-04   dRMS = 4.27413E-04
SCF Iteration   4: Energy = -113.7287812472690831   dE = -9.22806E-05   dRMS = 3.23115E-04
SCF Iteration   5: Energy = -113.7288131295568689   dE = -3.18823E-05   dRMS = 2.18596E-04
SCF Iteration   6: Energy = -113.7288271434591280   dE = -1.40139E-05   dRMS = 1.67272E-04
SCF Iteration   7: Energy = -113.7288339957377872   dE = -6.85228E-06   dRMS = 1.18511E-04
SCF Iteration   8: Energy = -113.7288375348278322   dE = -3.53909E-06   dRMS = 8.95995E-05
SCF Iteration   9: En

SCF Iteration  33: Energy = -113.7155969999141831   dE = -2.27374E-13   dRMS = 1.87560E-08
SCF Iteration  34: Energy = -113.7155969999143821   dE = -1.98952E-13   dRMS = 1.33306E-08
SCF Iteration  35: Energy = -113.7155969999141405   dE =  2.41585E-13   dRMS = 9.46343E-09
SCF Iteration  36: Energy = -113.7155969999144247   dE = -2.84217E-13   dRMS = 6.72475E-09
SCF Iteration  37: Energy = -113.7155969999144389   dE = -1.42109E-14   dRMS = 4.77468E-09
SCF Iteration  38: Energy = -113.7155969999141405   dE =  2.98428E-13   dRMS = 3.39245E-09
SCF Iteration  39: Energy = -113.7155969999144531   dE = -3.12639E-13   dRMS = 2.40897E-09
SCF Iteration  40: Energy = -113.7155969999140837   dE =  3.69482E-13   dRMS = 1.71143E-09
SCF Iteration  41: Energy = -113.7155969999142826   dE = -1.98952E-13   dRMS = 1.21538E-09
SCF Iteration  42: Energy = -113.7155969999144247   dE = -1.42109E-13   dRMS = 8.63397E-10
SCF Iteration  43: Energy = -113.7155969999144105   dE =  1.42109E-14   dRMS = 6.13176E-10

SCF Iteration  19: Energy = -113.7155969973278360   dE = -2.54239E-09   dRMS = 2.24646E-06
SCF Iteration  20: Energy = -113.7155969986098114   dE = -1.28198E-09   dRMS = 1.60759E-06
SCF Iteration  21: Energy = -113.7155969992562490   dE = -6.46438E-10   dRMS = 1.13468E-06
SCF Iteration  22: Energy = -113.7155969995822886   dE = -3.26040E-10   dRMS = 8.10197E-07
SCF Iteration  23: Energy = -113.7155969997469640   dE = -1.64675E-10   dRMS = 5.72928E-07
SCF Iteration  24: Energy = -113.7155969998298701   dE = -8.29061E-11   dRMS = 4.08452E-07
SCF Iteration  25: Energy = -113.7155969998716358   dE = -4.17657E-11   dRMS = 2.89215E-07
SCF Iteration  26: Energy = -113.7155969998927816   dE = -2.11458E-11   dRMS = 2.05961E-07
SCF Iteration  27: Energy = -113.7155969999032550   dE = -1.04734E-11   dRMS = 1.45971E-07
SCF Iteration  28: Energy = -113.7155969999086551   dE = -5.40012E-12   dRMS = 1.03871E-07
SCF Iteration  29: Energy = -113.7155969999114120   dE = -2.75691E-12   dRMS = 7.36645E-08

In [10]:
print("cqed_cis energy relative to rhf for \lambda_z:", z_cqed_cis_dict['CQED-CIS ENERGY'][0]+z_cqed_rhf_dict['CQED-RHF ENERGY']-rhf_energy)
print("cqed_cis energy relative to rhf for \lambda_yz:", yz_cqed_cis_dict['CQED-CIS ENERGY'][0]+yz_cqed_rhf_dict['CQED-RHF ENERGY']-rhf_energy)


print("cqed_cis energy relative to cqed-rhf for \lambda_z:", z_cqed_cis_dict['CQED-CIS ENERGY'][0])
print("cqed_cis energy relative to cqed-rhf for \lambda_yz:", yz_cqed_cis_dict['CQED-CIS ENERGY'][0])


cqed_cis energy relative to rhf for \lambda_z: 0.12317963225071082
cqed_cis energy relative to rhf for \lambda_yz: 0.11633495303493646
cqed_cis energy relative to rhf for \lambda_z: -0.03844554445269451
cqed_cis energy relative to rhf for \lambda_yz: -0.03204563845471793


Now analyze the transitions that contribute to the CQED-CIS ground state
for the $\lambda_{yz}$ 

In [14]:

# get orbital energies for both of these polarizations
eps_yz = yz_cqed_rhf_dict["CQED-RHF EPS"]
eps_eV = 27.211 * eps_yz


# number of doubly occupied orbitals
ndocc   = wfn.nalpha()
    
# total number of orbitals
nmo     = wfn.nmo()

nvirt = nmo - ndocc
    
# Note yz-polarized field belongs to A' 
cs_irrep = ["1A'", "2A'", "3A'", "4A'", "5A'", "6A'", "1A''", "7A'", "2A''", "8A'", "9A'", "10A'", "3A''", 
            "11A'", "12A'"]


lam_yz_c = yz_cqed_cis_dict['CQED-CIS L VECTORS']
state=0
print(" excitation with lambda_yz")
for i in range(0, ndocc):
    for a in range(0, nvirt):
        A = a + ndocc
        ia0 = 2 * (i * nvirt + a) + 2
        ia1 = 2 * (i * nvirt + a) + 3
        amp00 = lam_yz_c[ia0,state] * np.conj(lam_yz_c[ia0,state])
        amp01 = lam_yz_c[ia1,state] * np.conj(lam_yz_c[ia1,state])
        if amp00 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";0",amp00," : ", cs_irrep[i], "->",cs_irrep[a+ndocc], eps_eV[i], eps_eV[a+ndocc])
        if amp01 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";1",amp01," : ", cs_irrep[i], "->",cs_irrep[a+ndocc], eps_eV[i], eps_eV[a+ndocc])
            


 excitation with lambda_yz
4 ;0-> 12 ;1 (0.0018402434390781789+0j)  :  4A' -> 10A' -24.68219879204259 14.13739433958213
5 ;0-> 11 ;1 (0.001079012280672516+0j)  :  5A' -> 9A' -19.031609235331867 11.667226615090334
5 ;0-> 12 ;1 (0.0015706687442300989+0j)  :  5A' -> 10A' -19.031609235331867 14.13739433958213
6 ;0-> 12 ;1 (0.0012075036905632142+0j)  :  6A' -> 10A' -17.552548001474403 14.13739433958213
7 ;0-> 9 ;1 (0.007276699487663493+0j)  :  1A'' -> 2A'' -13.841224806311812 4.660214591282355
8 ;0-> 12 ;1 (0.004365229223147301+0j)  :  7A' -> 10A' -11.497276666800742 14.13739433958213


Now analyze the transitions that contribute to the CQED-CIS ground state
for the $\lambda_{z}$ 

In [17]:
# results for z-polarized
# get orbital energies for both of these polarizations
eps_z = z_cqed_rhf_dict["CQED-RHF EPS"]
eps_eV = 27.211 * eps_z


# number of doubly occupied orbitals
ndocc   = wfn.nalpha()
    
# total number of orbitals
nmo     = wfn.nmo()

nvirt = nmo - ndocc


    
# Note y-polarized field belongs to B2 and z-polarized field belongs to A1
c2v_irrep = ["1A1", "2A1", "3A1", "4A1", "1B2", "5A1", "1B1", "2B2", 
         "2B1", "6A1", "3B2", "7A1","3B1", "8A1", "4B2", "5B2", "9A1"]

lam_z_c = z_cqed_cis_dict['CQED-CIS L VECTORS']
state=0
print(" excitation with lambda_z")
for i in range(0, ndocc):
    for a in range(0, nvirt):
        A = a + ndocc
        ia0 = 2 * (i * nvirt + a) + 2
        ia1 = 2 * (i * nvirt + a) + 3
        amp00 = lam_z_c[ia0,state] * np.conj(lam_z_c[ia0,state])
        amp01 = lam_z_c[ia1,state] * np.conj(lam_z_c[ia1,state])
        if amp00 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";0",amp00," : ", c2v_irrep[i], "->",c2v_irrep[a+ndocc], eps_eV[i], eps_eV[a+ndocc])
        if amp01 > 1e-3:
            print(i+1,";0->",a+1+ndocc,";1",amp01," : ", c2v_irrep[i], "->",c2v_irrep[a+ndocc], eps_eV[i], eps_eV[a+ndocc])
            

 excitation with lambda_z
4 ;0-> 12 ;1 (0.0012518927055941908+0j)  :  4A1 -> 7A1 -25.20766453938373 12.834979069189636
6 ;0-> 12 ;1 (0.0022482959088339456+0j)  :  5A1 -> 7A1 -17.967226469689866 12.834979069189636
7 ;0-> 9 ;0 (0.002919669914392131+0j)  :  1B1 -> 2B1 -13.717547268345339 4.628745257024189
7 ;0-> 9 ;1 (0.014922685056668421+0j)  :  1B1 -> 2B1 -13.717547268345339 4.628745257024189
8 ;0-> 11 ;1 (0.004886271564006125+0j)  :  2B2 -> 3B2 -11.725580429582353 11.915964556036556
